In [1]:
# https://www.kaggle.com/competitions/tabular-playground-series-sep-2022/code
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib  inline

In [22]:
train =  pd.read_csv('train.csv')
train

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240
...,...,...,...,...,...,...
70123,70123,2020-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614
70124,70124,2020-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,215
70125,70125,2020-12-31,Spain,KaggleRama,Kaggle Getting Started,158
70126,70126,2020-12-31,Spain,KaggleRama,Kaggle Recipe Book,135


In [23]:
test = pd.read_csv('test.csv')
test

,row_id,date,country,store,product
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques
...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle Advanced Techniques
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle Getting Started
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle Recipe Book


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70128 entries, 0 to 70127
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    70128 non-null  int64 
 1   date      70128 non-null  object
 2   country   70128 non-null  object
 3   store     70128 non-null  object
 4   product   70128 non-null  object
 5   num_sold  70128 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.2+ MB


In [5]:
train['country'].unique()

array(['Belgium', 'France', 'Germany', 'Italy', 'Poland', 'Spain'],
      dtype=object)

In [6]:
train['store'].unique()

array(['KaggleMart', 'KaggleRama'], dtype=object)

In [7]:
train['product'].unique()

array(['Kaggle Advanced Techniques', 'Kaggle Getting Started',
       'Kaggle Recipe Book', 'Kaggle for Kids: One Smart Goose'],
      dtype=object)

In [25]:
traindata = train['num_sold']
test_id = test['row_id']

train.drop(['row_id', 'num_sold'], axis=1, inplace=True)
test.drop('row_id', axis=1, inplace=True)
# train と test を結合したほうがいい
data0 = pd.concat([train, test], axis=0).reset_index(drop=True)
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     87648 non-null  object
 1   country  87648 non-null  object
 2   store    87648 non-null  object
 3   product  87648 non-null  object
dtypes: object(4)
memory usage: 2.7+ MB


In [13]:
# https://deepage.net/features/pandas-to-datetime.html
pd.to_datetime(data0['date'])
data0.info()

<class 'pandas.core.series.Series'>
RangeIndex: 87648 entries, 0 to 87647
Series name: date
Non-Null Count  Dtype         
--------------  -----         
87648 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 684.9 KB


In [28]:
# seperate the data into year, month and day
data0['year'] = data0['date'].apply(lambda x: x.split('-')[0])
data0['month'] = data0['date'].apply(lambda x: x.split('-')[1])
data0['day'] = data0['date'].apply(lambda x: x.split('-')[2])
data0.drop('date', axis=1, inplace=True)

KeyError: 'date'

In [30]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  87648 non-null  object
 1   store    87648 non-null  object
 2   product  87648 non-null  object
 3   year     87648 non-null  object
 4   month    87648 non-null  object
 5   day      87648 non-null  object
dtypes: object(6)
memory usage: 4.0+ MB


In [31]:
# change the data types of year, month and day
data0[['year', 'month', 'day']] = data0[['year', 'month', 'day']].astype('int')
data0.dtypes
# .value_counts()

country    object
store      object
product    object
year        int32
month       int32
day         int32
dtype: object

In [ ]:
# create new feature 'season'
def get_season(month):
    '''
        return 0 for months 1-3,
            1 for months 4-6,
            2 for months 7-9,
            3 otherwise
    '''
    m = month
    if m in [1, 2, 3]: return 0
    elif m in [4, 5, 6]: return 1
    elif m in [7, 8, 9]: return 2
    else: return 3
train_test_X['season'] = train_test_X['month'].apply(get_season)
train_test_X.head()

In [33]:
pd.crosstab(
    index=data0['country'],
    columns=data0['store']
)
# そのデータを抽出したいときは？

store,KaggleMart,KaggleRama
country,,
Belgium,7304,7304
France,7304,7304
Germany,7304,7304
Italy,7304,7304
Poland,7304,7304
Spain,7304,7304


In [35]:
num_sold_month = data0.groupby('month')['num_sold'].sum()
num_sold_month

KeyError: 'Column not found: num_sold'

In [ ]:
print(data0.shape)
data1 = pd.get_dummies(data0)
print(data1.shape)

In [ ]:
# 標準化
s = StandardScaler()
s.fit(data1)
data2 = pd.DataFrame(s.transform(data1), index=data1.index, columns=data1.columns)

In [ ]:
# データ分割
train_final = data2.loc[:train.index.max(), :].copy()
test_final = data2.loc[train.index.max() + 1:, :].reset_index(drop=True).copy()